---

## Random Forest

---

### Qu'est-ce qu'un Random Forest ? 
Un **Random Forest** est un modèle basé sur des arbres de décision. Il combine plusieurs arbres indépendants pour :
- Améliorer la précision.
- Réduire le risque de surapprentissage.

---

### Pourquoi utiliser Random Forest pour les ventes en magasin ? 
1. **Intègre des données variées :**
   - Historique des ventes, promotions, jours fériés, météo, etc.
2. **Modélise les relations complexes :**
   - Les ventes dépendent souvent de facteurs non linéaires (ex. : promotions).
3. **Résiste au bruit et aux données manquantes :**
   - Idéal pour des données imparfaites.

---

### Limites pour les séries temporelles
- Ne capture pas naturellement les tendances ou la saisonnalité.
- Besoin de créer manuellement des caractéristiques temporelles (lags, jours de la semaine, etc.).


In [14]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import plotly.express as px
from useful_functions import *
file_path = "./results.json"

In [15]:
# --Tables--
series_train = pd.read_csv("series_train.csv")
series_test = pd.read_csv("series_test.csv")

In [16]:
vars = ['holiday','oil_price','onpromotion']

In [17]:
X_train = series_train[vars] # Exclure les colonnes inutiles pour l'entraînement
y_train = series_train['sales']

X_test = series_test[vars] # Exclure les colonnes inutiles pour les prédictions
y_test = series_test['sales']

# girdsearch
Trouvons les meilleurs parametres


In [18]:
param_grid = {
    'n_estimators': [50, 100, 200],         # Nombre d'arbres dans la forêt
    'max_depth': [5, 10, 20],              # Profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],       # Nombre minimum d'échantillons pour diviser un nœud
    'min_samples_leaf': [1, 2, 4]          # Nombre minimum d'échantillons dans une feuille
}

grid_search_RF = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,                                  # Validation croisée (5-fold)
    scoring='neg_mean_squared_error',      
    verbose=1,                             
    n_jobs=-1                              # Utilisation maximale des CPU disponibles
)


grid_search_RF.fit(X_train, y_train)

print("Meilleurs paramètres trouvés :")
print(grid_search_RF.best_params_)


best_RF_model = grid_search_RF.best_estimator_

# --predictions train--

y_train_pred = best_RF_model.predict(X_train)
series_train['train_Forecast_RF'] = y_train_pred

# --predictions test--

y_test_pred = best_RF_model.predict(X_test)
series_test['test_Forecast_RF'] = y_test_pred


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Meilleurs paramètres trouvés :
{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}


In [19]:
fig = px.line(series_train, x='date', y=['sales', 'train_Forecast_RF'],title="Predictions du train")
fig.show()
fig = px.line(series_test, x='date', y=['sales', 'test_Forecast_RF'],title="Predictions du test")
fig.show()

In [20]:
nrmse, mape, mae,r2 = calculate_metrics(y_test, y_test_pred)
print(nrmse)
print(mape)
print(mae)
print(r2)

0.11107951935278812
1.0271937783582502e+20
286934.02743705566
-0.505255277380146


```python
save_model_results(
    file_path=file_path,
    model_name="Random_forest",
    params={'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50},
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)
```

In [21]:
#series_train.to_csv("series_train",index =False, header=True)
#series_test.to_csv("series_test",index =False, header=True)